### En este Jupyter Notebook se realizan las siguientes tareas:
#### - Importación de los eventos de los 4 partidos, transformación y concatenación en un mismo DF para su exportación y posterior carga en SQL
#### - Cálculo de métricas de los partidos (métricas de eventing y tracking) y exportación para su posterior carga en SQL
#### - Cálculo de los minutos totales jugados por cada jugador y exporación para su posterior visualización.

### Importación de librerías y funciones

In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import numpy as np
import warnings
warnings.filterwarnings('ignore')

from funciones import *

### Importación de datos de eventos de partidos. 
#### 4 partidos de España en el mundial: ESP-CRI, ESP-ALE, ESP-JAP Y ESP-MAR

In [2]:
df_esp_cri=pd.read_csv(r'C:\Users\david\OneDrive\Documentos\DATA ANALYTICS\Proyectos\Github\FP_Fifa-World-Cup-2022_Spain-Analysis\data\20221123_FWC_10_ESP-CRC_FIFA_Unified_Event_Data_Events.csv')
df_esp_ale=pd.read_csv(r'C:\Users\david\OneDrive\Documentos\DATA ANALYTICS\Proyectos\Github\FP_Fifa-World-Cup-2022_Spain-Analysis\data\20221127_FWC_28_ESP-GER_FIFA_Unified_Event_Data_Events (1).csv')
df_esp_jap=pd.read_csv(r'C:\Users\david\OneDrive\Documentos\DATA ANALYTICS\Proyectos\Github\FP_Fifa-World-Cup-2022_Spain-Analysis\data\20221201_FWC_43_JPN-ESP_FIFA_Unified_Event_Data_Events.csv')
df_esp_mar=pd.read_csv(r'C:\Users\david\OneDrive\Documentos\DATA ANALYTICS\Proyectos\Github\FP_Fifa-World-Cup-2022_Spain-Analysis\data\20221206_FWC_55_MAR-ESP_FIFA_Unified_Event_Data_Events.csv')

### Transformación inicial y concatenación 

In [3]:
lista_eventos_partidos = [df_esp_cri, df_esp_ale, df_esp_jap, df_esp_mar]

In [4]:
### Creación de nueva columna de tiempo de partido en segundos, en cada uno de los 4 dfs.

for i in lista_eventos_partidos:
    i['match_time_in_min'] = i.match_time_in_ms.apply(pasar_ms_a_min)

In [5]:
### Concatenación de los 4 dfs.

df_ev = pd.concat([df_esp_cri, df_esp_ale, df_esp_jap, df_esp_mar], axis=0)

### Exportación de  DF concatenado de eventos para SQL y Visualización

In [6]:
#df_ev.to_excel(r'C:\Users\david\Proyectos\QATAR 2022\PROYECTO ESPAÑA\repositorio\datos_para_sql\tabla_eventos.xlsx', index=False)

In [7]:
#df_ev.to_excel(r'C:\Users\david\Proyectos\QATAR 2022\PROYECTO ESPAÑA\repositorio\viz\ficheros Tableau\tabla_eventos.xlsx', index=False)

### CÁLCULO DE MÉTRICAS

### Cálculo de métricas de eventos de los partidos

#### Se llama a la función 'métricas_partido' (en funciones.py) para el cálculo de las métricas de cada partido (más de 100 métricas por equipo y partido)

In [8]:
est_esp_cri = metricas_partido(df_esp_cri)
est_esp_ale = metricas_partido(df_esp_ale)
est_esp_jap = metricas_partido(df_esp_jap)
est_esp_mar = metricas_partido(df_esp_mar)

In [9]:
df_est = pd.concat([est_esp_cri, est_esp_ale, est_esp_jap, est_esp_mar], axis=0)
df_est

,match_id,team,corners,faltas_lanzadas,faltas_directas_lanzadas,faltas_realizadas,penaltis,penaltis_marcados,%_acierto_penaltis,saques_banda,saques_banda_recepcionados,%_acierto_sb,ofrecimientos,ofrecimientos_con_movimiento,ofrecimientos_con_movimiento_camp_contr,ofrecimientos_con_movimiento_ult_terc,remates_totales,remates_totales_fa,remates_on_target,remates_on_target_fa,remates_off_target,%_remates_on_target,goles,goles_fa,%_gol/on_target,goles_concedidos,centros_totales,centros_completados,asistencias_centro,pases,pases_completados,%_acierto_pases,pases_completados_linebreak,asistencias_pase,cambios_de_juego,pases_ultimo_tercio,pases_completados_ultimo_tercio,progresiones,intentos_regate,intentos_regate_ult_terc,regates_completados,regates_completados_ult_terc,recepciones,recep_behind,recep_3_4,recep_2_3,recep_1_2,recep_inside,%_recep_inside,recep_outside,%_recep_outside,recep_ultimo_tercio,bloqueos,despejes,intercepciones,entradas_totales,entradas_exitosas,%_entradas_exitosas,take_on_against,pushing-on,presiones,presiones_directas,%_presion_directa,gk_active_eng,gk_aerial_control,gk_def_support,gk_goalprevention_tot,gk_goalprevention_tot_saved,duelos_totales,duelos_totales_ganados,%_duelos_totales_ganados,duelos_campo_ganados,duelos_físicos_ganados,duelos_aereos_ganados,sustituciones,posesion,posesión_disputa,pases_verticales,pases_adelante,pases_atras,pases_horizontales,%_pases_verticales,%_pases_adelante,%_pases_atras,%_pases_horizontales,recuperaciones,recuperaciones_campo_contr,%_recuperaciones_campo_contr,recuperaciones_campo_prop,%_recuperaciones_campo_prop,recuperaciones1,recuperaciones_campo_contr1,%_recuperaciones_campo_contr1,recuperaciones_campo_prop1,%_recuperaciones_campo_prop1,perdidas,perdidas_campo_prop,%_perdidas_campo_prop,tarj_amarillas,tarjetas_rojas
0,133004,SPAIN,5,18,12,11,1,1,100.0,24,24,100.0,1677,1085,694,309,17,3,7,0,8,41.2,7,0,100.0,0,17,2,2,1037,976,94.1,193,2,8,203,180,42,11,9,2,2,1092,23,0,60,307,628,57.5,464,42.5,232,16,10,4,18,7,38.9,4,208,185,50,27.0,3,1,1,0,0,37,19,51,6,2,11,5,78.81,1.96,87,153,136,687,9,16,14,70,38,18,47,20,53,47,20,43,27,57,116,26,22,0,0
1,133004,COSTA RICA,0,11,8,18,0,0,none,13,11,84.6,409,250,103,43,0,0,0,0,8,0.0,0,0,0.0,7,5,0,0,218,159,72.9,43,0,1,21,16,9,4,2,2,1,226,1,1,23,74,139,61.5,87,38.5,31,21,30,8,42,13,31.0,11,795,585,47,8.0,7,0,7,17,10,37,18,49,1,1,16,5,19.23,1.96,22,37,27,95,14,23,17,60,38,7,18,31,82,39,7,18,32,82,104,42,40,2,0
0,133021,SPAIN,6,16,14,15,0,0,none,23,21,91.3,1040,633,282,112,7,2,3,1,2,42.9,1,0,33.3,1,9,2,1,624,542,86.9,118,0,9,69,55,31,11,5,1,0,666,16,3,47,186,362,54.4,304,45.6,92,24,28,6,40,13,32.5,9,316,275,45,16.4,5,4,3,11,9,33,16,48,6,1,9,4,60.23,2.17,53,98,88,356,10,18,16,66,47,13,28,34,72,50,15,30,35,70,139,62,45,1,0
1,133021,GERMANY,5,15,13,16,0,0,none,15,8,53.3,670,440,209,107,10,3,4,0,2,40.0,1,0,25.0,1,13,1,0,337,270,80.1,63,0,2,53,30,30,9,4,4,1,379,9,2,34,132,246,64.9,133,35.1,82,32,24,8,45,14,31.1,11,508,387,59,15.2,3,0,2,7,6,36,18,50,8,1,9,5,37.60,2.17,35,59,42,169,13,22,16,63,48,15,31,33,69,53,16,30,37,70,128,32,25,3,0
0,133035,SPAIN,2,11,9,8,0,0,none,22,20,90.9,1747,1138,825,451,14,7,5,1,2,35.7,1,0,20.0,2,20,2,0,1048,969,92.5,126,1,8,244,218,25,10,6,6,3,1092,17,0,18,193,674,61.7,418,38.3,282,7,15,11,16,8,50.0,4,173,150,25,16.7,1,0,1,6,4,30,13,43,3,0,10,5,78.95,1.58,59,123,123,723,6,13,13,75,35,11,31,24,69,35,11,31,24,69,128,19,15,0,0
1,133035,JAPAN,0,8,6,11,0,0,none,11,10,90.9,437,315,123,56,6,2,3,1,2,50.0,2,1,66.7,1,6,3,1,214,155,72.4,45,1,2,29,26,10,4,2,2,1,237,10,0,21,75,159,67.1,78,32.9,44,39,39,10,29,6,20.7,10,791,637,62,9.7,3,1,1,14,13,30,17,57,4,1,12,5,19.47,1.58,27,44,20,91,17,28,13,59,34,4,12,30,88,34,4,12,30,88,91,34,37,3,0
0,128074,SPAIN,8,20,14,18,3,0,0.0,33,33,100.0,1621,896,779,446,16,3,3,0,6,18.8,0,0,0.0,3,27,4,0,1008,932,92.5,126,0,8,196,167,26,13,8,4,1,1075,27,0,26,165,697,64.8,378,35.2,245,19,26,7,41,5,12.2,11,222,217,56,25.8,1,1,0,10,6,47,23,49,4,2,17,6,71.35,2.31,96,142,79,711,10,15,8,76,33,12,36,21,64,36,13

In [10]:
### Se calcula la métrica 'asistencias totales' por la suma de 'asistencias centro' + 'asistencias pase'

df_est['asistencias_totales'] = df_est.asistencias_centro + df_est.asistencias_pase

In [11]:
### Se incorpora la métrica xG (goles esperados) obtenida de Internet (FBREF.com)

xG = [3.5, 0, 0.6, 1.3, 1.4, 1, 0.7, 1]
df_est['xG'] = xG

In [12]:
df_est

,match_id,team,corners,faltas_lanzadas,faltas_directas_lanzadas,faltas_realizadas,penaltis,penaltis_marcados,%_acierto_penaltis,saques_banda,saques_banda_recepcionados,%_acierto_sb,ofrecimientos,ofrecimientos_con_movimiento,ofrecimientos_con_movimiento_camp_contr,ofrecimientos_con_movimiento_ult_terc,remates_totales,remates_totales_fa,remates_on_target,remates_on_target_fa,remates_off_target,%_remates_on_target,goles,goles_fa,%_gol/on_target,goles_concedidos,centros_totales,centros_completados,asistencias_centro,pases,pases_completados,%_acierto_pases,pases_completados_linebreak,asistencias_pase,cambios_de_juego,pases_ultimo_tercio,pases_completados_ultimo_tercio,progresiones,intentos_regate,intentos_regate_ult_terc,regates_completados,regates_completados_ult_terc,recepciones,recep_behind,recep_3_4,recep_2_3,recep_1_2,recep_inside,%_recep_inside,recep_outside,%_recep_outside,recep_ultimo_tercio,bloqueos,despejes,intercepciones,entradas_totales,entradas_exitosas,%_entradas_exitosas,take_on_against,pushing-on,presiones,presiones_directas,%_presion_directa,gk_active_eng,gk_aerial_control,gk_def_support,gk_goalprevention_tot,gk_goalprevention_tot_saved,duelos_totales,duelos_totales_ganados,%_duelos_totales_ganados,duelos_campo_ganados,duelos_físicos_ganados,duelos_aereos_ganados,sustituciones,posesion,posesión_disputa,pases_verticales,pases_adelante,pases_atras,pases_horizontales,%_pases_verticales,%_pases_adelante,%_pases_atras,%_pases_horizontales,recuperaciones,recuperaciones_campo_contr,%_recuperaciones_campo_contr,recuperaciones_campo_prop,%_recuperaciones_campo_prop,recuperaciones1,recuperaciones_campo_contr1,%_recuperaciones_campo_contr1,recuperaciones_campo_prop1,%_recuperaciones_campo_prop1,perdidas,perdidas_campo_prop,%_perdidas_campo_prop,tarj_amarillas,tarjetas_rojas,asistencias_totales,xG
0,133004,SPAIN,5,18,12,11,1,1,100.0,24,24,100.0,1677,1085,694,309,17,3,7,0,8,41.2,7,0,100.0,0,17,2,2,1037,976,94.1,193,2,8,203,180,42,11,9,2,2,1092,23,0,60,307,628,57.5,464,42.5,232,16,10,4,18,7,38.9,4,208,185,50,27.0,3,1,1,0,0,37,19,51,6,2,11,5,78.81,1.96,87,153,136,687,9,16,14,70,38,18,47,20,53,47,20,43,27,57,116,26,22,0,0,4,3.5
1,133004,COSTA RICA,0,11,8,18,0,0,none,13,11,84.6,409,250,103,43,0,0,0,0,8,0.0,0,0,0.0,7,5,0,0,218,159,72.9,43,0,1,21,16,9,4,2,2,1,226,1,1,23,74,139,61.5,87,38.5,31,21,30,8,42,13,31.0,11,795,585,47,8.0,7,0,7,17,10,37,18,49,1,1,16,5,19.23,1.96,22,37,27,95,14,23,17,60,38,7,18,31,82,39,7,18,32,82,104,42,40,2,0,0,0.0
0,133021,SPAIN,6,16,14,15,0,0,none,23,21,91.3,1040,633,282,112,7,2,3,1,2,42.9,1,0,33.3,1,9,2,1,624,542,86.9,118,0,9,69,55,31,11,5,1,0,666,16,3,47,186,362,54.4,304,45.6,92,24,28,6,40,13,32.5,9,316,275,45,16.4,5,4,3,11,9,33,16,48,6,1,9,4,60.23,2.17,53,98,88,356,10,18,16,66,47,13,28,34,72,50,15,30,35,70,139,62,45,1,0,1,0.6
1,133021,GERMANY,5,15,13,16,0,0,none,15,8,53.3,670,440,209,107,10,3,4,0,2,40.0,1,0,25.0,1,13,1,0,337,270,80.1,63,0,2,53,30,30,9,4,4,1,379,9,2,34,132,246,64.9,133,35.1,82,32,24,8,45,14,31.1,11,508,387,59,15.2,3,0,2,7,6,36,18,50,8,1,9,5,37.60,2.17,35,59,42,169,13,22,16,63,48,15,31,33,69,53,16,30,37,70,128,32,25,3,0,0,1.3
0,133035,SPAIN,2,11,9,8,0,0,none,22,20,90.9,1747,1138,825,451,14,7,5,1,2,35.7,1,0,20.0,2,20,2,0,1048,969,92.5,126,1,8,244,218,25,10,6,6,3,1092,17,0,18,193,674,61.7,418,38.3,282,7,15,11,16,8,50.0,4,173,150,25,16.7,1,0,1,6,4,30,13,43,3,0,10,5,78.95,1.58,59,123,123,723,6,13,13,75,35,11,31,24,69,35,11,31,24,69,128,19,15,0,0,1,1.4
1,133035,JAPAN,0,8,6,11,0,0,none,11,10,90.9,437,315,123,56,6,2,3,1,2,50.0,2,1,66.7,1,6,3,1,214,155,72.4,45,1,2,29,26,10,4,2,2,1,237,10,0,21,75,159,67.1,78,32.9,44,39,39,10,29,6,20.7,10,791,637,62,9.7,3,1,1,14,13,30,17,57,4,1,12,5,19.47,1.58,27,44,20,91,17,28,13,59,34,4,12,30,88,34,4,12,30,88,91,34,37,3,0,2,1.0
0,128074,SPAIN,8,20,14,18,3,0,0.0,33,33,100.0,1621,896,779,446,16,3,3,0,6,18.8,0,0,0.0,3,27,4,0,1008,932,92.5,126,0,8,196,167,26,13,8,4,1,1075,27,0,26,165,697,64.8,378,35.2,245,19,26,7,41,5,12.2,11,222,217,56,25.8,1,1,0,10,6,47,23,49,4,2,17

### Importación de los datos de tracking generales

In [13]:
df_tr_esp_cri = pd.read_excel(r'C:\Users\david\OneDrive\Documentos\DATA ANALYTICS\Proyectos\Github\FP_Fifa-World-Cup-2022_Spain-Analysis\data\Tracking Data Metrics M-10.xlsx')
df_tr_esp_ale = pd.read_excel(r'C:\Users\david\OneDrive\Documentos\DATA ANALYTICS\Proyectos\Github\FP_Fifa-World-Cup-2022_Spain-Analysis\data\Tracking Data Metrics M-28 (1).xlsx')
df_tr_esp_jap = pd.read_excel(r'C:\Users\david\OneDrive\Documentos\DATA ANALYTICS\Proyectos\Github\FP_Fifa-World-Cup-2022_Spain-Analysis\data\Tracking Data Metrics M-43.xlsx')
df_tr_esp_mar = pd.read_excel(r'C:\Users\david\OneDrive\Documentos\DATA ANALYTICS\Proyectos\Github\FP_Fifa-World-Cup-2022_Spain-Analysis\data\Tracking Data Metrics M-55.xlsx')

### Cálculo de métricas de tracking

#### Se llama a la función 'métricas_tracking_partido' (en funciones.py) para el cálculo 

In [14]:
est_tr_esp_cri = metricas_tracking_partido(df_tr_esp_cri)
est_tr_esp_ale = metricas_tracking_partido(df_tr_esp_ale)
est_tr_esp_jap = metricas_tracking_partido(df_tr_esp_jap)
est_tr_esp_mar = metricas_tracking_partido(df_tr_esp_mar)

### Concatenación de tablas con las métricas de tracking

In [15]:
df_est_tr = pd.concat([est_tr_esp_cri, est_tr_esp_ale, est_tr_esp_jap, est_tr_esp_mar], axis=0)
df_est_tr

,team,distancia_recorrida_km,n_speed_runs_(15-20km/h),n_of_sprints_(>20km/h),total_average_speed_(km/h)
0,Spain,122.43399,1331,548,6.720440
1,Costa Rica,116.61037,1455,453,6.505847
0,Spain,112.65189,1461,557,6.504642
1,Germany,115.54569,1559,617,6.681131
0,Spain,114.04544,1326,459,6.558801
1,Japan,116.00564,1333,493,6.866877
0,Spain,142.64314,1450,636,6.147745
1,Morocco,147.78556,1533,614,6.100761


In [16]:
#df_est.team = df_est.team.apply(capitalizar)

### Unión lateral de métricas de eventos y métricas de tracking

In [17]:
df_est = df_est.reset_index(drop=True)
df_est_tr = df_est_tr.reset_index(drop=True)

In [18]:
df_est['distancia_recorrida_km'] = df_est_tr['distancia_recorrida_km']
df_est['n_speed_runs_(15-20km/h)'] = df_est_tr['n_speed_runs_(15-20km/h)']
df_est['n_of_sprints_(>20km/h)'] = df_est_tr['n_of_sprints_(>20km/h)']
df_est['total_average_speed_(km/h)'] = df_est_tr['total_average_speed_(km/h)']

In [19]:
#df_est.set_index('team').join(df_est_tr.set_index('team'))

In [20]:
#pd.merge(df_est, df_est_tr, how='left', on=['team', 'team'])

### Tabla final de métricas calculadas

In [21]:
df_est

,match_id,team,corners,faltas_lanzadas,faltas_directas_lanzadas,faltas_realizadas,penaltis,penaltis_marcados,%_acierto_penaltis,saques_banda,saques_banda_recepcionados,%_acierto_sb,ofrecimientos,ofrecimientos_con_movimiento,ofrecimientos_con_movimiento_camp_contr,ofrecimientos_con_movimiento_ult_terc,remates_totales,remates_totales_fa,remates_on_target,remates_on_target_fa,remates_off_target,%_remates_on_target,goles,goles_fa,%_gol/on_target,goles_concedidos,centros_totales,centros_completados,asistencias_centro,pases,pases_completados,%_acierto_pases,pases_completados_linebreak,asistencias_pase,cambios_de_juego,pases_ultimo_tercio,pases_completados_ultimo_tercio,progresiones,intentos_regate,intentos_regate_ult_terc,regates_completados,regates_completados_ult_terc,recepciones,recep_behind,recep_3_4,recep_2_3,recep_1_2,recep_inside,%_recep_inside,recep_outside,%_recep_outside,recep_ultimo_tercio,bloqueos,despejes,intercepciones,entradas_totales,entradas_exitosas,%_entradas_exitosas,take_on_against,pushing-on,presiones,presiones_directas,%_presion_directa,gk_active_eng,gk_aerial_control,gk_def_support,gk_goalprevention_tot,gk_goalprevention_tot_saved,duelos_totales,duelos_totales_ganados,%_duelos_totales_ganados,duelos_campo_ganados,duelos_físicos_ganados,duelos_aereos_ganados,sustituciones,posesion,posesión_disputa,pases_verticales,pases_adelante,pases_atras,pases_horizontales,%_pases_verticales,%_pases_adelante,%_pases_atras,%_pases_horizontales,recuperaciones,recuperaciones_campo_contr,%_recuperaciones_campo_contr,recuperaciones_campo_prop,%_recuperaciones_campo_prop,recuperaciones1,recuperaciones_campo_contr1,%_recuperaciones_campo_contr1,recuperaciones_campo_prop1,%_recuperaciones_campo_prop1,perdidas,perdidas_campo_prop,%_perdidas_campo_prop,tarj_amarillas,tarjetas_rojas,asistencias_totales,xG,distancia_recorrida_km,n_speed_runs_(15-20km/h),n_of_sprints_(>20km/h),total_average_speed_(km/h)
0,133004,SPAIN,5,18,12,11,1,1,100.0,24,24,100.0,1677,1085,694,309,17,3,7,0,8,41.2,7,0,100.0,0,17,2,2,1037,976,94.1,193,2,8,203,180,42,11,9,2,2,1092,23,0,60,307,628,57.5,464,42.5,232,16,10,4,18,7,38.9,4,208,185,50,27.0,3,1,1,0,0,37,19,51,6,2,11,5,78.81,1.96,87,153,136,687,9,16,14,70,38,18,47,20,53,47,20,43,27,57,116,26,22,0,0,4,3.5,122.43399,1331,548,6.720440
1,133004,COSTA RICA,0,11,8,18,0,0,none,13,11,84.6,409,250,103,43,0,0,0,0,8,0.0,0,0,0.0,7,5,0,0,218,159,72.9,43,0,1,21,16,9,4,2,2,1,226,1,1,23,74,139,61.5,87,38.5,31,21,30,8,42,13,31.0,11,795,585,47,8.0,7,0,7,17,10,37,18,49,1,1,16,5,19.23,1.96,22,37,27,95,14,23,17,60,38,7,18,31,82,39,7,18,32,82,104,42,40,2,0,0,0.0,116.61037,1455,453,6.505847
2,133021,SPAIN,6,16,14,15,0,0,none,23,21,91.3,1040,633,282,112,7,2,3,1,2,42.9,1,0,33.3,1,9,2,1,624,542,86.9,118,0,9,69,55,31,11,5,1,0,666,16,3,47,186,362,54.4,304,45.6,92,24,28,6,40,13,32.5,9,316,275,45,16.4,5,4,3,11,9,33,16,48,6,1,9,4,60.23,2.17,53,98,88,356,10,18,16,66,47,13,28,34,72,50,15,30,35,70,139,62,45,1,0,1,0.6,112.65189,1461,557,6.504642
3,133021,GERMANY,5,15,13,16,0,0,none,15,8,53.3,670,440,209,107,10,3,4,0,2,40.0,1,0,25.0,1,13,1,0,337,270,80.1,63,0,2,53,30,30,9,4,4,1,379,9,2,34,132,246,64.9,133,35.1,82,32,24,8,45,14,31.1,11,508,387,59,15.2,3,0,2,7,6,36,18,50,8,1,9,5,37.60,2.17,35,59,42,169,13,22,16,63,48,15,31,33,69,53,16,30,37,70,128,32,25,3,0,0,1.3,115.54569,1559,617,6.681131
4,133035,SPAIN,2,11,9,8,0,0,none,22,20,90.9,1747,1138,825,451,14,7,5,1,2,35.7,1,0,20.0,2,20,2,0,1048,969,92.5,126,1,8,244,218,25,10,6,6,3,1092,17,0,18,193,674,61.7,418,38.3,282,7,15,11,16,8,50.0,4,173,150,25,16.7,1,0,1,6,4,30,13,43,3,0,10,5,78.95,1.58,59,123,123,723,6,13,13,75,35,11,31,24,69,35,11,31,24,69,128,19,15,0,0,1,1.4,114.04544,1326,459,6.558801
5,133035,JAPAN,0,8,6,11,0,0,none,11,10,90.9,437,315,123,56,6,2,3,1,2,50.0,2,1,66.7,1,6,3,1,214,155,72.4,45,1,2,29,26,10,4,2,2,1,237,10,0,21,75,159,67.1,78,32.9,44,39,39,10,29,6,20.7,10,791,637,62,9.7,3,1,1,14,13,30,17,57,4,1,12,5,19.47,1.58,27,44,20,91,17,28,13,59,34,4,12,30,88,34,4,12,30,88,91,34,37,3,0,2,1.0,11

### Exportación de  DF_est (métricas calculadas) para SQL y Visualización)

In [22]:
#df_est.to_excel(r'C:\Users\david\Proyectos\QATAR 2022\PROYECTO ESPAÑA\repositorio\datos_para_sql\tabla_metricas.xlsx', index=False)

In [23]:
#df_est.to_excel(r'C:\Users\david\Proyectos\QATAR 2022\PROYECTO ESPAÑA\repositorio\viz\ficheros Tableau\tabla_metricas.xlsx', index=False)

### CÁLCULO DE MINUTOS JUGADOS DE CADA JUGADOR

### Importación de tabla jugadores y preparación de la misma

In [24]:
df_pl=pd.read_excel(r'C:\Users\david\OneDrive\Documentos\DATA ANALYTICS\Proyectos\Github\FP_Fifa-World-Cup-2022_Spain-Analysis\viz\ficheros Tableau\players.xlsx')
df_pl.head()

,competition_id,team_id,team,player_id,player_shirt_number,player_position,player_name,player_first_name,player_last_name,player_date_birth,team_code
0,255711,43901,Costa Rica,199175,1,goalkeeper,Keylor NAVAS,Keilor Antonio,NAVAS GAMBOA,1986-12-15,CRC
1,255711,43901,Costa Rica,405824,2,midfielder,Daniel CHACON,Daniel Alonso,CHACÓN SALAS,2001-04-11,CRC
2,255711,43901,Costa Rica,411350,3,defender,Juan Pablo VARGAS,Juan Pablo,VARGAS CAMPOS,1995-06-06,CRC
3,255711,43901,Costa Rica,439979,4,defender,Keysher FULLER,Keysher,FULLER SPENCE,1994-07-12,CRC
4,255711,43901,Costa Rica,236530,5,midfielder,Celso BORGES,Celso,BORGES MORA,1988-05-27,CRC


In [25]:
df_minutos = df_pl[df_pl.team=='Spain'][['player_id', 'player_name']]

In [26]:
df_minutos['minutos_jugados'] = 0
df_minutos

,player_id,player_name,minutos_jugados
104,430736,Robert SANCHEZ,0
105,270948,Cesar AZPILICUETA,0
106,405562,Eric GARCIA,0
107,430721,Pau TORRES,0
108,303034,Sergio BUSQUETS,0
109,382739,Marcos LLORENTE,0
110,314291,Alvaro MORATA,0
111,313410,KOKE,0
112,447866,GAVI,0
113,400715,Marco ASENSIO,0


### Cálculo de minutos jugados totales por jugador

In [27]:
minutos_jugados(df_esp_cri, df_minutos)
minutos_jugados(df_esp_ale, df_minutos)
minutos_jugados(df_esp_jap, df_minutos)
minutos_jugados(df_esp_mar, df_minutos)

,player_id,player_name,minutos_jugados
104,430736,Robert SANCHEZ,0
105,270948,Cesar AZPILICUETA,134
106,405562,Eric GARCIA,0
107,430721,Pau TORRES,90
108,303034,Sergio BUSQUETS,363
109,382739,Marcos LLORENTE,120
110,314291,Alvaro MORATA,185
111,313410,KOKE,52
112,447866,GAVI,284
113,400715,Marco ASENSIO,229


### Exportación de  DF_minutos jugados para Visualización)

In [28]:
#df_minutos.to_excel(r'C:\Users\david\Proyectos\QATAR 2022\PROYECTO ESPAÑA\repositorio\viz\ficheros Tableau\tabla_minutos_jugados.xlsx', index=False)